# 7.4. 含并行连结的网络（GoogLeNet）
:label:`sec_googlenet`

在2014年的ImageNet图像识别挑战赛中，一个名叫*GoogLeNet* :cite:`Szegedy.Liu.Jia.ea.2015` 的网络结构大放异彩。
GoogLeNet吸收了NiN中串联网络的思想，并在此基础上做了改进。
这篇论文的一个重点是解决了什么样大小的卷积核最合适的问题。
毕竟，以前流行的网络使用小到 $1 \times 1$ ，大到 $11 \times 11$ 的卷积核。
本文的一个观点是，有时使用不同大小的卷积核组合是有利的。
在本节中，我们将介绍一个稍微简化的GoogLeNet版本：我们省略了一些为稳定训练而添加的特殊特性，但是现在有了更好的训练算法，这些特性不是必要的。


## (**7.4.1. Inception块**)

在GoogLeNet中，基本的卷积块被称为*Inception块*（Inception block）。这很可能得名于电影《盗梦空间》（Inception），因为电影中的一句话“我们需要走得更深”（“We need to go deeper”）。

![Inception块的结构。](../img/inception.svg)
:label:`fig_inception`

如 :numref:`fig_inception` 所示，Inception块由四条并行路径组成。
前三条路径使用窗口大小为 $1\times 1$、$3\times 3$ 和 $5\times 5$ 的卷积层，从不同空间大小中提取信息。
中间的两条路径在输入上执行 $1\times 1$ 卷积，以减少通道数，从而降低模型的复杂性。
第四条路径使用 $3\times 3$ 最大池化层，然后使用 $1\times 1$ 卷积层来改变通道数。
这四条路径都使用合适的填充来使输入与输出的高和宽一致，最后我们将每条线路的输出在通道维度上连结，并构成Inception块的输出。在Inception块中，通常调整的超参数是每层输出通道的数量。


In [3]:
import paddle
import paddle.nn as nn
import numpy as np

class Inception(nn.Layer):
    # c1 - c4为每条线路里的层的输出通道数
    def __init__(self, num_channels, c1, c2, c3, c4):
        super(Inception, self).__init__()
        # 线路1，单1 x 1卷积层
        p1 = [
            nn.Conv2D(num_channels, c1, 1),
            nn.ReLU()
        ]
        self.p1 = nn.Sequential(*p1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        p2 = [
            nn.Conv2D(num_channels, c2[0], 1),
            nn.ReLU(),
            nn.Conv2D(c2[0], c2[1], 3, padding=1),
            nn.ReLU()
        ]
        self.p2 = nn.Sequential(*p2)
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        p3 = [
            nn.Conv2D(num_channels, c3[0], 1),
            nn.ReLU(),
            nn.Conv2D(c3[0], c3[1], 5, padding=2),
            nn.ReLU()
        ]
        self.p3 = nn.Sequential(*p3)
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        p4 = [
            nn.MaxPool2D(3, stride=1, padding=1),
            nn.Conv2D(num_channels, c4, 1),
            nn.ReLU()
        ]
        self.p4 = nn.Sequential(*p4)
    def forward(self, X):
        # 在通道维上连结输出
        return paddle.concat([self.p1(X), self.p2(X), self.p3(X), self.p4(X)], axis=1)

那么为什么GoogLeNet这个网络如此有效呢？
首先我们考虑一下滤波器（filter）的组合，它们可以用各种滤波器尺寸探索图像，这意味着不同大小的滤波器可以有效地识别不同范围的图像细节。
同时，我们可以为不同的滤波器分配不同数量的参数。


## [**7.4.2. GoogLeNet模型**]

如 :numref:`fig_inception_full` 所示，GoogLeNet 一共使用 9 个Inception块和全局平均池化层的堆叠来生成其估计值。Inception块之间的最大池化层可降低维度。
第一个模块类似于 AlexNet 和 LeNet，Inception块的栈从VGG继承，全局平均池化层避免了在最后使用全连接层。

![GoogLeNet结构。](../img/inception-full.svg)
:label:`fig_inception_full`

现在，我们逐一实现GoogLeNet的每个模块。第一个模块使用 64 个通道、 $7\times 7$ 卷积层。


In [4]:
model = [
            nn.Conv2D(1, 64, 7, stride=2, padding=3),
            nn.ReLU(),
            nn.MaxPool2D(kernel_size=3, stride=2, padding=1)
]

第二个模块使用两个卷积层：第一个卷积层是 64个通道、 $1\times 1$ 卷积层；第二个卷积层使用将通道数量增加三倍的 $3\times 3$ 卷积层。
这对应于 Inception 块中的第二条路径。


In [5]:
model += [
             nn.Conv2D(64, 64, 1),
             nn.ReLU(),
             nn.Conv2D(64, 192, 3, padding=1),
             nn.ReLU(),
             nn.MaxPool2D(3, stride=2, padding=1)
         ]

第三个模块串联两个完整的Inception块。
第一个 Inception 块的输出通道数为 $64+128+32+32=256$，四个路径之间的输出通道数量比为 $64:128:32:32=2:4:1:1$。
第二个和第三个路径首先将输入通道的数量分别减少到 $96/192=1/2$ 和 $16/192=1/12$，然后连接第二个卷积层。第二个 Inception 块的输出通道数增加到 $128+192+96+64=480$，四个路径之间的输出通道数量比为 $128:192:96:64 = 4:6:3:2$。
第二条和第三条路径首先将输入通道的数量分别减少到 $128/256=1/2$ 和 $32/256=1/8$。


In [6]:
model += [
            Inception(64, 64, (96, 128), (16, 32), 32),
            # Inception(192, 64, (96, 128), (16, 32), 32),
            Inception(256, 128, (128, 192), (32, 96), 64),
            nn.MaxPool2D(3, stride=2, padding=1)
        ]

第四模块更加复杂，
它串联了5个Inception块，其输出通道数分别是 $192+208+48+64=512$ 、 $160+224+64+64=512$ 、 $128+256+64+64=512$ 、 $112+288+64+64=528$ 和 $256+320+128+128=832$ 。
这些路径的通道数分配和第三模块中的类似，首先是含 $3×3$ 卷积层的第二条路径输出最多通道，其次是仅含 $1×1$ 卷积层的第一条路径，之后是含 $5×5$ 卷积层的第三条路径和含 $3×3$ 最大池化层的第四条路径。
其中第二、第三条路径都会先按比例减小通道数。
这些比例在各个 Inception 块中都略有不同。


In [7]:
model += [
            Inception(480, 192, (96, 208), (16, 48), 64),
            Inception(512, 160, (112, 224), (24, 64), 64),
            Inception(512, 128, (128, 256), (24, 64), 64),
            Inception(512, 112, (144, 288), (32, 64), 64),
            Inception(528, 256, (160, 320), (32, 128), 128),
            nn.MaxPool2D(3, stride=2, padding=1)
        ]

第五模块包含输出通道数为 $256+320+128+128=832$ 和 $384+384+128+128=1024$ 的两个Inception块。
其中每条路径通道数的分配思路和第三、第四模块中的一致，只是在具体数值上有所不同。
需要注意的是，第五模块的后面紧跟输出层，该模块同 NiN 一样使用全局平均池化层，将每个通道的高和宽变成1。
最后我们将输出变成二维数组，再接上一个输出个数为标签类别数的全连接层。


In [8]:
model += [
            Inception(832, 256, (160, 320), (32, 128), 128),
            Inception(832, 384, (192, 384), (48, 128), 128),
            paddle.fluid.dygraph.Pool2D(pool_type='max', global_pooling=True)
        ]

GoogLeNet 模型的计算复杂，而且不如 VGG 那样便于修改通道数。
[**为了使Fashion-MNIST上的训练短小精悍，我们将输入的高和宽从224降到96**]，这简化了计算。下面演示各个模块输出的形状变化。


In [9]:
class GoogleNet(nn.Layer):
    def __init__(self, num_classes=10):
        super(GoogleNet, self).__init__()
        # GoogLeNet跟VGG一样，在主体卷积部分中使用5个模块（block），
        # 个模块之间使用步幅为2的3×3最大池化层来减小输出高宽。
        # 第一模块使用一个64通道的7×7卷积层。
        model = [
            nn.Conv2D(1, 64, 7, stride=2, padding=3),
            nn.ReLU(),
            nn.MaxPool2D(3, stride=2, padding=1)
        ]
        # 第二模块使用2个卷积层：首先是64通道的1×1卷积层，
        # 然后是将通道增大3倍的3×3卷积层。
        # 它对应Inception块中的第二条线路。
        # model += [
        #     nn.Conv2D(64, 64, 1),
        #     nn.ReLU(),
        #     nn.Conv2D(64, 192, 3, padding=1),
        #     nn.ReLU(),
        #     nn.MaxPool2D(3, stride=2, padding=1)
        # ]
        # 第三模块串联2个完整的Inception块。
        # 第一个Inception块的输出通道数为64+128+32+32=256，
        # 其中4条线路的输出通道数比例为64:128:32:32=2:4:1:1。
        # 其中第二、第三条线路先分别将输入通道数减小至96/192=1/2和16/192=1/12后，再接上第二层卷积层。
        # 第二个Inception块输出通道数增至128+192+96+64=480，
        # 每条线路的输出通道数之比为128:192:96:64=4:6:3:2。
        # 其中第二、第三条线路先分别将输入通道数减小至128/256=1/2和32/256=1/8。
        model += [
            Inception(64, 64, (96, 128), (16, 32), 32),
            # Inception(192, 64, (96, 128), (16, 32), 32),
            Inception(256, 128, (128, 192), (32, 96), 64),
            nn.MaxPool2D(3, stride=2, padding=1)
        ]
        # 第四模块更加复杂。它串联了5个Inception块，
        # 其输出通道数分别是192+208+48+64=512、160+224+64+64=512、
        # 128+256+64+64=512、112+288+64+64=528528和256+320+128+128=832。
        # 这些线路的通道数分配和第三模块中的类似，
        # 首先是含3×3卷积层的第二条线路输出最多通道，
        # 其次是仅含1×1卷积层的第一条线路，
        # 之后是含5×5卷积层的第三条线路和含3×3最大池化层的第四条线路。
        # 其中第二、第三条线路都会先按比例减小通道数。
        # 这些比例在各个Inception块中都略有不同。
        model += [
            Inception(480, 192, (96, 208), (16, 48), 64),
            Inception(512, 160, (112, 224), (24, 64), 64),
            Inception(512, 128, (128, 256), (24, 64), 64),
            Inception(512, 112, (144, 288), (32, 64), 64),
            Inception(528, 256, (160, 320), (32, 128), 128),
            nn.MaxPool2D(3, stride=2, padding=1)
        ]
        # 第五模块有输出通道数为256+320+128+128=832和384+384+128+128=1024的两个Inception块。
        # 其中每条线路的通道数的分配思路和第三、第四模块中的一致，只是在具体数值上有所不同。
        # 需要注意的是，第五模块的后面紧跟输出层，该模块同NiN一样使用全局平均池化层来将每个通道的高和宽变成1。
        # 最后我们将输出变成二维数组后接上一个输出个数为标签类别数的全连接层。
        model += [
            Inception(832, 256, (160, 320), (32, 128), 128),
            Inception(832, 384, (192, 384), (48, 128), 128),
            paddle.fluid.dygraph.Pool2D(pool_type='max', global_pooling=True)
        ]
        self.model = nn.Sequential(*model)
        self.fc = nn.Sequential(nn.Linear(1024, 10))
    def forward(self, X):
        Y = self.model(X)
        Y = paddle.flatten(Y, start_axis=1)
        Y = self.fc(Y)
        return Y

with paddle.fluid.dygraph.guard():
    googlenet = GoogleNet()
    X = paddle.to_tensor(np.random.uniform(-1, 1, [1, 1, 28, 28]).astype('float32'))
    Y = googlenet(X)
    print(Y.shape)

[1, 10]


GoogLeNet模型的计算复杂，而且不如VGG那样便于修改通道数。本节里我们将输入的高和宽从224降到96来简化计算。下面演示各个模块之间的输出的形状变化。

In [10]:
with paddle.fluid.dygraph.guard():
    googlenet = GoogleNet(10)
    param_info = paddle.summary(googlenet, (1, 1, 28, 28))
    print(param_info)

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1       [[1, 1, 28, 28]]     [1, 64, 14, 14]         3,200     
    ReLU-1       [[1, 64, 14, 14]]     [1, 64, 14, 14]           0       
  MaxPool2D-1    [[1, 64, 14, 14]]      [1, 64, 7, 7]            0       
   Conv2D-2       [[1, 64, 7, 7]]       [1, 64, 7, 7]          4,160     
    ReLU-2        [[1, 64, 7, 7]]       [1, 64, 7, 7]            0       
   Conv2D-3       [[1, 64, 7, 7]]       [1, 96, 7, 7]          6,240     
    ReLU-3        [[1, 96, 7, 7]]       [1, 96, 7, 7]            0       
   Conv2D-4       [[1, 96, 7, 7]]       [1, 128, 7, 7]        110,720    
    ReLU-4        [[1, 128, 7, 7]]      [1, 128, 7, 7]           0       
   Conv2D-5       [[1, 64, 7, 7]]       [1, 16, 7, 7]          1,040     
    ReLU-5        [[1, 16, 7, 7]]       [1, 16, 7, 7]            0       
   Conv2D-6       [[1, 16, 7, 7]]   

## [**7.4.3. 训练模型**]

和以前一样，我们使用 Fashion-MNIST 数据集来训练我们的模型。在训练之前，我们将图片转换为 $96 \times 96$ 分辨率。


In [11]:
import paddle
import paddle.vision.transforms as T
from paddle.vision.datasets import FashionMNIST

# 数据集处理
transform = T.Compose([
    T.Resize(96),
    T.Transpose(),
    T.Normalize([127.5], [127.5]),
])
train_dataset = FashionMNIST(mode='train', transform=transform)
val_dataset = FashionMNIST(mode='test', transform=transform)
# 模型定义
model = paddle.Model(GoogleNet(10))
# 设置训练模型所需的optimizer, loss, metric
model.prepare(
    paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters()),
    paddle.nn.CrossEntropyLoss(),
    paddle.metric.Accuracy(topk=(1, 5)))
# 启动训练、评估
model.fit(train_dataset, val_dataset, epochs=2, batch_size=64, log_freq=100)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/2
step 100/938 - loss: 0.7295 - acc_top1: 0.4831 - acc_top5: 0.8505 - 60ms/step
step 200/938 - loss: 0.4486 - acc_top1: 0.6225 - acc_top5: 0.9213 - 61ms/step


## 7.4.4. 小结

* Inception 块相当于一个有4条路径的子网络。它通过不同窗口形状的卷积层和最大池化层来并行抽取信息，并使用 $1×1$ 卷积层减少每像素级别上的通道维数从而降低模型复杂度。
*  GoogLeNet将多个设计精细的Inception块与其他层（卷积层、全连接层）串联起来。其中Inception块的通道数分配之比是在 ImageNet 数据集上通过大量的实验得来的。
* GoogLeNet 和它的后继者们一度是 ImageNet 上最有效的模型之一：它以较低的计算复杂度提供了类似的测试精度。


## 7.4.5. 练习

1. GoogLeNet 有数个后续版本。尝试实现并运行它们，然后观察实验结果。这些后续版本包括：
    * 添加批量归一化层 :cite:`Ioffe.Szegedy.2015`（batch normalization），在 :numref:`sec_batch_norm`中将介绍）。
    * 对 Inception 模块进行调整。
    * 使用标签平滑（label smoothing）进行模型正则化 :cite:`Szegedy.Vanhoucke.Ioffe.ea.2016`。
    * 加入残差连接 :cite:`Szegedy.Ioffe.Vanhoucke.ea.2017` ，（ :numref:`sec_resnet` 将介绍）。
1. 使用 GoogLeNet 的最小图像大小是多少？
1. 将 AlexNet、VGG 和 NiN 的模型参数大小与 GoogLeNet 进行比较。后两个网络结构是如何显著减少模型参数大小的？


[Discussions](https://discuss.d2l.ai/t/1871)
